# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import csv
import json 
import ipympl
# %matplotlib
from pylab import rcParams
import math
import requests

In [ ]:
q=requests.get("https://diseasex-backend.herokuapp.com/outbreaks")
while(q.text!="[]"):
    q= requests.delete("https://diseasex-backend.herokuapp.com/outbreaks")
    q=requests.get("https://diseasex-backend.herokuapp.com/outbreaks")

In [ ]:
clusteringDB=requests.get("https://diseasex-backend.herokuapp.com/humanCases")
clusteringDB.status_code

In [ ]:
json_data = clusteringDB.json()
# print(type(json_data))
# print(json_data)

In [ ]:
with open('humanCase.json', 'w') as json_file:
    json.dump(json_data, json_file)

## Converting JSON to CSV

In [ ]:
import json  # For JSON loading
import csv  # For CSV dict writer


def get_leaves(item, key=None, key_prefix=""):
    
    if isinstance(item, dict):
        leaves = {}
        for item_key in item.keys():
            temp_key_prefix = (
                item_key if (key_prefix == "") else (key_prefix + "_" + str(item_key))
            )
            leaves.update(get_leaves(item[item_key], item_key, temp_key_prefix))
        return leaves
    elif isinstance(item, list):
        leaves = {}
        elements = []
        for element in item:
            if isinstance(element, dict) or isinstance(element, list):
                leaves.update(get_leaves(element, key, key_prefix))
            else:
                elements.append(element)
        if len(elements) > 0:
            leaves[key] = elements
        return leaves
    else:
        return {key_prefix: item}


with open("humanCase.json") as f_input, open("final.csv", "w", newline="") as f_output:
    json_data = json.load(f_input, strict=False)
    fieldnames = set()
    for entry in json_data:
        fieldnames.update(get_leaves(entry).keys())
    csv_output = csv.DictWriter(f_output, delimiter=",", fieldnames=sorted(fieldnames))
    csv_output.writeheader()
    csv_output.writerows(get_leaves(entry) for entry in json_data)

## Reading Data

In [ ]:
df=pd.read_csv('final.csv',engine='python')
df
# df.dropna(axis=0,how='any',subset=['lat','long'],inplace=True)

In [ ]:
df.dropna(subset=['lat', 'lng'],inplace=True)

In [ ]:
# X=df.loc[:,['disease_name','lat','lng']]
X = df
X

In [ ]:
count = X['disease_name'].nunique()
# df['hID'].nunique()
count

In [ ]:
X['disease'] = X['disease_name']

## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
dfle = X
dfle.disease = le.fit_transform(dfle.disease)
dfle.keys()

In [ ]:
dfle['disease_precautions'][1]

In [ ]:
count = dfle['disease'].nunique()

In [ ]:
# ax = dfle.plot.scatter(x='lat', y='long', color='DarkGreen',s = 0);
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
scatter = ax.scatter(dfle['lat'],dfle['lng'],color='blue',s=50)

In [ ]:
my_df = dfle.copy()
my_df = my_df.iloc[0:0]
my_df

In [ ]:
centers = [0,0]
centers
print(type(centers))

## K-Means Clustering

In [ ]:
for x in range(count):
    xdf = dfle.loc[dfle['disease'] == x]
#     print(xdf)
    dist_points_from_cluster_center = []
    q = xdf.shape[0]
    temp = xdf.loc[:,['lat','lng']]
    K = range(1,q)
    if(q>1):
        for no_of_clusters in K:
            k_model = KMeans(n_clusters = no_of_clusters)
            k_model.fit(temp)
            dist_points_from_cluster_center.append(k_model.inertia_)

        #Function to find distance between a point and a line
        def calc_distance(x1,y1,a,b,c):
            d=abs((a*x1 + b*y1 +c))/(math.sqrt(a*a + b*b))
            return d
        print(dist_points_from_cluster_center)
        a = dist_points_from_cluster_center[0] - dist_points_from_cluster_center[q-2]
        b = K[q-2] - K[0]
        c1 = K[0]*dist_points_from_cluster_center[q-2]
        c2 = K[q-2]*dist_points_from_cluster_center[0]
        c = c1-c2

        distance_of_points_from_line = []
        for k in range(q-1):
            distance_of_points_from_line.append(
            calc_distance(K[k], dist_points_from_cluster_center[k],a,b,c))

        optimum_k = distance_of_points_from_line.index(max(distance_of_points_from_line)) + 1
    else:
        optimum_k = 1
    kmeans = KMeans(n_clusters = optimum_k, init ='k-means++',random_state=50)

    kmeans.fit(temp)
    y = kmeans.fit_predict(temp)
    
    xdf = xdf.copy()
    xdf['cluster_label'] = (y + (x*10))/10
    
    center = kmeans.cluster_centers_
#     print(type(center))
    xdf['cluster_centre_lat'] = center[0,0]
    xdf['cluster_centre_lng'] = center[0,1]
#     xdf['center'] = center
#     print(centers)
#     print(center)
    centers = np.vstack((centers,center))
    xdf
    my_df = my_df.append(xdf,ignore_index=True,sort = False)

Labelled Dataframe

In [ ]:
my_df.fillna(0,inplace=True)
my_df

# Centers

In [ ]:
# centers.remove([0,0])
centers = np.delete(centers, 0,0) 
centers

In [ ]:
labels = my_df['cluster_label']
len(labels)

In [ ]:
from collections import Counter, defaultdict
print(Counter(labels))

# Visualising Points

In [ ]:
# plt.style.use(['dark_background'])
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
scatter = ax.scatter(my_df['lat'],my_df['lng'],c=labels,s=50)
for i,j in centers:
    ax.scatter(i,j,s=100,c='red',marker='+')
ax.set_xlabel('lat')
ax.set_ylabel('lng')
# plt.colorbar(scatter)

plt.show()

In [ ]:
import json

data = centers
list = [{"latitude": x[0], "longitude": x[1]} for x in data]

output = json.dumps(list)

In [ ]:
output

In [ ]:
with open('result.json', 'w') as outfile:
    json.dump(output, outfile,indent = 4)

In [ ]:
import json
for i in range(my_df.shape[0]):
    ini_string={
        
        "disease": {
            "_id": my_df["disease__id"][i],
            "livestock": [
                {
                    "_id": my_df["disease_livestock__id"][i],
                    "breed": my_df["disease_livestock_breed"][i],
                    "population": my_df["disease_livestock_population"][i]
                }
            ],
            "vaccine": [
                {
                    "_id": my_df["disease_vaccine__id"][i],
                    "name": my_df["disease_vaccine_name"][i],
                    "scientificName": my_df["disease_vaccine_scientificName"][i],
                    "duration": my_df["disease_vaccine_duration"][i],
                    "forHuman": my_df["disease_vaccine_forHuman"][i]
                }
            ],
            "name": my_df["disease_name"][i],
            "scientificName": my_df["disease_scientificName"][i],
            "precautions" : my_df["disease_precautions"][i],
            "symptoms": my_df["disease_symptoms"][i],
            "morbidity": my_df["disease_morbidity"][i],
            "mortality": my_df["disease_mortality"][i],
            "total_affected": my_df["disease_total_affected"][i],
            "total_deaths": my_df["disease_total_deaths"][i]
        },
        "healthCenter": {
            "_id": my_df["healthCenter__id"][i],
            "address": my_df["healthCenter_address"][i],
            "email": my_df["healthCenter_email"][i],
            "contact": my_df["healthCenter_contact"][i],
            "name": my_df["healthCenter_contact"][i],
            "incharge": my_df["healthCenter_incharge"][i],
            "pincode": my_df["healthCenter_pincode"][i],
            "web": my_df["healthCenter_web"][i],
            "total_recovered":my_df["healthCenter_total_recovered"][i],
            "total_affected": my_df["healthCenter_total_affected"][i],
            "total_deaths": my_df["healthCenter_total_deaths"][i]
            
        },
        "total_recovered":my_df["disease_total_recovered"][i],
        "total_affected": my_df["disease_total_affected"][i],
        "total_deaths": my_df["disease_total_deaths"][i],
        "__v": my_df["__v"][i],
        "radius" :my_df["disease"][i],
        "lat": my_df["cluster_centre_lat"][i],
        "lng": my_df["cluster_centre_lng"][i]
    }

    def convert(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()

    y=json.dumps(ini_string, default=convert)
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json'}
    url="https://diseasex-backend.herokuapp.com/outbreaks"
    r = requests.post(url, data=y,headers=headers)    

In [ ]:
print("Clustering Successful !")
print("Data upload to database Successful !")